In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [19]:
# Obtiene resumen de egresos por GRD, filtrando solamente las hospitalizaciones
df = (
    (
        pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
        .filter(
            (pl.col("COD_HOSPITAL") == 112103)
            & (pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN")
            & (pl.col("ANIO_EGRESO") <= 2020)
        )
        .with_columns(pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X"))
        .group_by(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .agg(
            [
                pl.col("CIP_ENCRIPTADO").n_unique().alias("n_pacientes_distintos"),
                pl.len().alias("n_egresos"),
                pl.col("IR_29301_PESO").mean().round(4).alias("peso_grd_promedio"),
                pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
            ]
        )
        .sort(by=["ANIO_EGRESO", "n_egresos"], descending=True)
    )
    .collect(streaming=True)
    .to_pandas()
)

In [20]:
resumen_egresos_torax_grd = pd.pivot_table(
    df,
    index="DIAGNOSTICO1",
    columns=["ANIO_EGRESO"],
    values=["n_pacientes_distintos", "n_egresos", "dias_estada_totales", "peso_grd_promedio"],
    aggfunc="sum",
    fill_value=0,
)

In [21]:
resumen_egresos_torax_grd.to_excel("../data/interim/resumen_egresos_INT_grd.xlsx")

In [22]:
df_relevantes_deis = pd.read_excel(
    "../data/external/resumen_diagnosticos_relevantes_INT_largo.xlsx", index_col=False
)



In [29]:
resumen_egresos_torax_deis = pd.pivot_table(
    df_relevantes_deis,
    index="DIAG1",
    columns=["ANO_EGRESO"],
    values=[
        "ranking_nacionales_n_egresos",
        "n_pacientes_distintos",
        "n_egresos",
        "total_nacionales_n_egresos",
        "%_nacionales_n_egresos",
        "dias_estada_totales",
    ],
    aggfunc="sum",
    fill_value=0,
).sort_values(
    by=[("ranking_nacionales_n_egresos", 2019), ("n_egresos", 2019)],
    ascending=[True, False],
)[
    [
        "dias_estada_totales",
        "n_pacientes_distintos",
        "n_egresos",
        "total_nacionales_n_egresos",
        "%_nacionales_n_egresos",
        "ranking_nacionales_n_egresos",
    ]
]

In [31]:
resumen_mas_relevantes_con_peso = resumen_egresos_torax_deis.merge(
    resumen_egresos_torax_grd,
    how="left",
    left_index=True,
    right_index=True,
    suffixes=("_DEIS", "_GRD"),
)

In [32]:
resumen_mas_relevantes_con_peso

dias_estada_totales_DEIS                         \
                          2016  2017  2018  2019 2020   
DIAG1                                                   
C341                      2002  2360  2379  2767  974   
Z450                       150   141   243   190   78   
J841                      1600  1564  1286  1799  930   
C343                      1417  1446  1368  1095  427   
D381                       604   262   465   713  304   
...                        ...   ...   ...   ...  ...   
G908                         5     0     6     4    2   
R048                        10    22    13    23    0   
D142                         6     0     3    15    0   
D174                         0     0     7    20    0   
D862                         0     0    15    30    0   

      n_pacientes_distintos_DEIS                      ...  \
                            2016 2017 2018 2019 2020  ...   
DIAG1                                                 ...   
C341                         140  139  169  176   70  ...   
Z450                          98   70  127  131   56  ...   
J841                         104   96   96  102   53  ...   
C343                          98  100   96   90   36  ...   
D381                          69   32   71   80   38  ...   
...                          ...  ...  ...  ...  ...  ...   
G908                           3    0    3    2    1  ...   
R048                           1    2    1    2    0  ...   
D142                           1    0    1    1    0  ...   
D174                           0    0    2    1    0  ...   
D862                           0    0    1    1    0  ...   

      ranking_nacionales_n_egresos      dias_estada_totales_GRD         \
                              2019 2020                    2019   2020   
DIAG1                                                                    
C341                             1    1                  2756.0  944.0   
Z450                             1    2                   163.0   86.0   
J841                             1    1                  1813.0  930.0   
C343                             1    2                  1105.0  427.0   
D381                             1    1                   571.0  303.0   
...                            ...  ...                     ...    ...   
G908                             2    2                     4.0    2.0   
R048                             2    0                    23.0    0.0   
D142                             2    0                    15.0    0.0   
D174                             2    0                    20.0    0.0   
D862                             2    0                    30.0    0.0   

      n_egresos_GRD       n_pacientes_distintos_GRD       peso_grd_promedio  \
               2019  2020                      2019  2020              2019   
DIAG1                                                                         
C341          218.0  75.0                     176.0  70.0            1.5883   
Z450          137.0  59.0                     131.0  58.0            2.3587   
J841          123.0  58.0                     103.0  55.0            1.5129   
C343          122.0  38.0                      91.0  36.0            1.4067   
D381           83.0  46.0                      70.0  39.0            0.9234   
...             ...   ...                       ...   ...               ...   
G908            2.0   1.0                       2.0   1.0            1.5652   
R048            2.0   0.0                       2.0   0.0            1.3396   
D142            1.0   0.0                       1.0   0.0            2.1123   
D174            1.0   0.0                       1.0   0.0            2.1123   
D862            1.0   0.0                       1.0   0.0            1.0532   

               
         2020  
DIAG1          
C341   1.5971  
Z450   2.2691  
J841   1.4141  
C343   1.4022  
D381   0.8338  
...       ...  
G908   1.5652  
R048   0.0000  
D142   0.0000  
D174   0.0000  
